In [1]:
import cv2
import math
import cvzone
from ultralytics import YOLO
import pandas as pd
import yaml
from collections import defaultdict
from sklearn.metrics import accuracy_score

In [2]:
waste_categories = {
    "Recyclable": {
        "Dry Waste": ["aluminum", "cardboard", "paper", "glass bottle", "plastic bag", "plastic bottle", "plastic container", "plastic straw"],
        "E-Waste": ["Keyboard", "Mobile Phone", "Monitor"]
    },
    "Non-Recyclable": {
        "Wet Waste": ["egg shell", "facemask", "food wrapper", "fruit peels", "treeleaves", "vegetable peels"]
    }
}

In [3]:
# Load YOLO model
model = YOLO("D:/Waste_Detection_New/best_0.pt")

In [4]:
# Load video
video_path = "D:/Waste_Detection_New/Waste_detect.mp4"
cap = cv2.VideoCapture(video_path)

In [5]:
# Initialize waste summary dictionary
waste_summary = {"Recyclable": {"Dry Waste": defaultdict(int), "E-Waste": defaultdict(int)}, "Non-Recyclable": {"Wet Waste": defaultdict(int)}}
object_tracker = {}  # Track detected objects using their positions
next_object_id = 0  # ID counter for unique objects

In [6]:
# Accuracy calculation variables
correct_predictions = 0
total_predictions = 0
frame_idx = 0

In [7]:
def get_center(box):
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    return (x1 + x2) // 2, (y1 + y2) // 2

In [8]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv8 detection
    results = model(frame)
    predicted_labels = []
    new_objects = {}
    
    for r in results:
        for box in r.boxes:
            class_id = int(box.cls[0])
            label = model.names[class_id]
            conf = box.conf[0].item()
            center = get_center(box)
            predicted_labels.append(label)
            
            # Check if object is new or already tracked
            found = False
            for obj_id, prev_center in object_tracker.items():
                if isinstance(prev_center, tuple) and len(prev_center) == 2:
                    if abs(center[0] - prev_center[0]) < 50 and abs(center[1] - prev_center[1]) < 50:
                        object_tracker[obj_id] = center  # Ensure (x, y) tuple is stored
                        found = True
                        break
            
            if not found:
                new_objects[next_object_id] = (center[0], center[1])  # Store as tuple
                for category, subcategories in waste_categories.items():
                    for subcategory, items in subcategories.items():
                        if label in items:
                            waste_summary[category][subcategory][label] += 1
                next_object_id += 1
            
            # Draw bounding box and label on frame
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{label} {conf:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Set first frame as ground truth
    if frame_idx == 0:
        object_tracker = {label: (0, 0) for label in predicted_labels}  # Ensure tuples
    
    # Compare current frame predictions with tracked objects
    for label in predicted_labels:
        if label in object_tracker:
            correct_predictions += 1
        total_predictions += 1
    
    frame_idx += 1
    object_tracker.update(new_objects)
    
    # Show detection frame
    cv2.imshow("Waste Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 1 cardboard, 6 plastic bags, 513.2ms
Speed: 10.0ms preprocess, 513.2ms inference, 6.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cardboards, 7 plastic bags, 425.7ms
Speed: 6.3ms preprocess, 425.7ms inference, 8.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cardboards, 8 plastic bags, 427.2ms
Speed: 4.0ms preprocess, 427.2ms inference, 7.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 cardboard, 8 plastic bags, 399.9ms
Speed: 3.0ms preprocess, 399.9ms inference, 8.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 cardboard, 7 plastic bags, 408.7ms
Speed: 3.0ms preprocess, 408.7ms inference, 9.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 plastic bags, 437.8ms
Speed: 3.0ms preprocess, 437.8ms inference, 7.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 cardboard, 9 plastic bags, 413.2ms
Speed: 3.0ms preprocess, 413.2ms inference, 8.0ms postprocess per image at sha

In [9]:
# Calculate accuracy
video_accuracy = (correct_predictions / total_predictions) * 100 if total_predictions > 0 else 0

In [10]:
# Save accuracy to CSV
accuracy_data = pd.DataFrame({'Metric': ['Video Accuracy'], 'Value': [video_accuracy]})
accuracy_data.to_csv("video_model_accuracy.csv", index=False)

In [11]:
# Generate structured summary table
summary_list = []
for category, subcategories in waste_summary.items():
    for subcategory, items in subcategories.items():
        for item, count in items.items():
            summary_list.append([category, subcategory, item, count])

In [12]:
waste_df = pd.DataFrame(summary_list, columns=["Category", "Subcategory", "Waste Type", "Count"])

# Append accuracy to summary
accuracy_row = pd.DataFrame([["Model Performance", "", "Accuracy", video_accuracy]], columns=["Category", "Subcategory", "Waste Type", "Count"])
waste_df = pd.concat([waste_df, accuracy_row], ignore_index=True)
waste_df.to_csv("waste_summary.csv", index=False)

# Print final structured summary
print("Final Waste Classification Summary:")
print(waste_df)
print("Video Model Accuracy:", video_accuracy, "%")

Final Waste Classification Summary:
            Category Subcategory       Waste Type      Count
0         Recyclable   Dry Waste      plastic bag  28.000000
1         Recyclable   Dry Waste        cardboard   3.000000
2     Non-Recyclable   Wet Waste  vegetable peels   2.000000
3  Model Performance                     Accuracy  92.032967
Video Model Accuracy: 92.03296703296702 %
